In [1]:
import os

from IPython import display
# import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import random as python_random

import tensorflow as tf
import tensorflow_hub as hub
from tensorflow.keras import regularizers

from lazypredict.Supervised import LazyClassifier

from sklearn.impute import SimpleImputer
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import Pipeline


yamnet_model = hub.load('https://tfhub.dev/google/yamnet/1') # if there's an "SavedModel file does not exist at:", delete that folder and rerun it
vggish_model = hub.load('https://tfhub.dev/google/vggish/1')

2023-03-10 14:44:52.317297: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  SSE4.1 SSE4.2 AVX AVX2 AVX512F AVX512_VNNI FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-03-10 14:44:56.474409: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  SSE4.1 SSE4.2 AVX AVX2 AVX512F AVX512_VNNI FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [2]:
df_all = pd.read_csv('../train_val_test_split/train_val_test_GoogleAudioSet.csv', index_col=0)
df_all

file         source  \
0      ../data/interim/GoogleAudioSet_unbalanced_list...    Google_city   
1      ../data/interim/GoogleAudioSet_unbalanced_list...    Google_city   
2      ../data/interim/GoogleAudioSet_unbalanced_list...  Google_nature   
3      ../data/interim/GoogleAudioSet_unbalanced_list...    Google_city   
4      ../data/interim/GoogleAudioSet_unbalanced_list...    Google_city   
...                                                  ...            ...   
13662  ../data/interim/GoogleAudioSet_unbalanced_list...    Google_city   
13663  ../data/interim/GoogleAudioSet_unbalanced_list...    Google_city   
13664  ../data/interim/GoogleAudioSet_unbalanced_list...    Google_city   
13665  ../data/interim/GoogleAudioSet_unbalanced_list...    Google_city   
13666  ../data/interim/GoogleAudioSet_unbalanced_list...    Google_city   

       category  weight  fold  
0             1       1     0  
1             1       1     8  
2             0       1     5  
3             1       1     1  
4             1       1     1  
...         ...     ...   ...  
13662         1       1     5  
13663         1       1     3  
13664         1       1     8  
13665         1       1     8  
13666         1       1     0  

[13667 rows x 5 columns]

In [3]:
def data_preprocessing(filename):
    import pickle
    
    file = open(filename, 'rb')
    output = pickle.load(file)
#     output = pd.read_pickle(file)
    wav_raw = output['y_aug']
    wav_bg = output['y_bg_aug']
    wav_fg = output['y_fg_aug']
    
    df_indices_raw = output['df_indices_aug']
    df_indices_fg = output['df_indices_fg_aug']
    df_indices_bg = output['df_indices_bg_aug']
    
    wt = output['wt']
    mps_raw = output['mps_aug']
    mps_fg = output['mps_fg_aug']
    mps_bg = output['mps_bg_aug']
    mps_raw = mps_raw[:,wt<=100].reshape(-1) # exclude the temporal modulation frequency >100 Hz
    mps_fg = mps_fg[:,wt<=100].reshape(-1)
    mps_bg = mps_bg[:,wt<=100].reshape(-1)
    
    file.close()

    # Extract YAMNet embeddings for each frame
    scores, embedding_tensor_raw, spectrogram = yamnet_model(wav_raw)
    embedding_tensor_raw = tf.reduce_mean(embedding_tensor_raw, axis=0).numpy()

    scores, embedding_tensor_bg, spectrogram = yamnet_model(wav_bg)
    embedding_tensor_bg = tf.reduce_mean(embedding_tensor_bg, axis=0).numpy()

    scores, embedding_tensor_fg, spectrogram = yamnet_model(wav_fg)
    embedding_tensor_fg = tf.reduce_mean(embedding_tensor_fg, axis=0).numpy()
    
    
    # Extract VGGish embeddings for each frame
    vggish_embedding_raw = tf.reduce_mean(vggish_model(wav_raw), axis=0).numpy()
    vggish_embedding_bg = tf.reduce_mean(vggish_model(wav_bg), axis=0).numpy()
    vggish_embedding_fg = tf.reduce_mean(vggish_model(wav_fg), axis=0).numpy()
   
    return embedding_tensor_raw, embedding_tensor_bg, embedding_tensor_fg, mps_raw, mps_bg, mps_fg, df_indices_raw, df_indices_bg, df_indices_fg, vggish_embedding_raw, vggish_embedding_bg, vggish_embedding_fg

In [6]:
import librosa
import panns_inference
from panns_inference import AudioTagging, SoundEventDetection, labels
import pickle
from librosa import resample
import time

start_time = time.time()


wav_raw_list = []
wav_bg_list = []
wav_fg_list = []
for index, row in df_all.iterrows():
    filename = '../data/interim/augmented_'+row['file'][16:-6]+'.pkl'
    file = open(filename, 'rb')
    output = pickle.load(file)
    wav_raw_list.append(np.pad(output['y_aug'], (0,160000-len(output['y_aug'])),'mean'))
    wav_bg_list.append(np.pad(output['y_bg_aug'], (0,160000-len(output['y_bg_aug'])),'mean'))
    wav_fg_list.append(np.pad(output['y_fg_aug'], (0,160000-len(output['y_fg_aug'])),'mean'))
    file.close()
    
wav_raw = resample(np.stack(wav_raw_list), orig_sr=16000, target_sr=32000, axis=1)
del wav_raw_list
wav_bg = resample(np.stack(wav_bg_list), orig_sr=16000, target_sr=32000, axis=1)
del wav_bg_list
wav_fg = resample(np.stack(wav_fg_list), orig_sr=16000, target_sr=32000, axis=1)
del wav_fg_list
    

n_file = 0
clipwise_output_raw_list = []
clipwise_output_bg_list = []
clipwise_output_fg_list = []
embedding_raw_list = []
embedding_bg_list = []
embedding_fg_list = []
file_step = 100

at = AudioTagging(checkpoint_path=None, device='cuda')
while n_file < len(df_all):
    print(str(n_file))
    (clipwise_output_raw, embedding_raw) = at.inference(wav_raw[n_file:min(n_file+file_step,len(df_all))])
    (clipwise_output_bg, embedding_bg) = at.inference(wav_bg[n_file:min(n_file+file_step,len(df_all))])
    (clipwise_output_fg, embedding_fg) = at.inference(wav_fg[n_file:min(n_file+file_step,len(df_all))])
    n_file += file_step
    
    clipwise_output_raw_list.append(clipwise_output_raw)
    clipwise_output_bg_list.append(clipwise_output_bg)
    clipwise_output_fg_list.append(clipwise_output_fg)
    embedding_raw_list.append(embedding_raw)
    embedding_bg_list.append(embedding_bg)
    embedding_fg_list.append(embedding_fg)

    
panns_clip_raw = np.concatenate(clipwise_output_raw_list, axis=0)
del clipwise_output_raw_list

panns_clip_bg = np.concatenate(clipwise_output_bg_list, axis=0)
del clipwise_output_bg_list

panns_clip_fg = np.concatenate(clipwise_output_fg_list, axis=0)
del clipwise_output_fg_list

panns_embedding_raw = np.concatenate(embedding_raw_list, axis=0)
del embedding_raw_list

panns_embedding_bg = np.concatenate(embedding_bg_list, axis=0)
del embedding_bg_list

panns_embedding_fg = np.concatenate(embedding_fg_list, axis=0)
del embedding_fg_list



print(time.time()-start_time)

Checkpoint path: /Users/andrewchang/panns_data/Cnn14_mAP=0.431.pth
Using CPU.
0
100
200
300
400
500
600
700
800
900
1000
1100
1200
1300
1400
1500
1600
1700
1800
1900
2000
2100
2200
2300
2400
2500
2600
2700
2800
2900
3000
3100
3200
3300
3400
3500
3600
3700
3800
3900
4000
4100
4200
4300
4400
4500
4600
4700
4800
4900
5000
5100
5200
5300
5400
5500
5600
5700
5800
5900
6000
6100
6200
6300
6400
6500
6600
6700
6800
6900
7000
7100
7200
7300
7400
7500
7600
7700
7800
7900
8000
8100
8200
8300
8400
8500
8600
8700
8800
8900
9000
9100
9200
9300
9400
9500
9600
9700
9800
9900
10000
10100
10200
10300
10400
10500
10600
10700
10800
10900
11000
11100
11200
11300
11400
11500
11600
11700
11800
11900
12000
12100
12200
12300
12400
12500
12600
12700
12800
12900
13000
13100
13200
13300
13400
13500
13600
13436.996054172516


In [7]:
np.save('../data/processed/20230310_dataAugmentation/panns_clip_raw.npy', panns_clip_raw)
np.save('../data/processed/20230310_dataAugmentation/panns_clip_bg.npy', panns_clip_bg)
np.save('../data/processed/20230310_dataAugmentation/panns_clip_fg.npy', panns_clip_fg)
np.save('../data/processed/20230310_dataAugmentation/panns_embedding_raw.npy', panns_embedding_raw)
np.save('../data/processed/20230310_dataAugmentation/panns_embedding_bg.npy', panns_embedding_bg)
np.save('../data/processed/20230310_dataAugmentation/panns_embedding_fg.npy', panns_embedding_fg)

In [8]:
embedding_raw_list = []
embedding_bg_list = []
embedding_fg_list = []
mps_raw_list = []
mps_bg_list = []
mps_fg_list = []
indices_raw_list = []
indices_bg_list = []
indices_fg_list = []
vgg_raw_list = []
vgg_bg_list = []
vgg_fg_list = []

import time
start_time = time.time()


for index, row in df_all.iterrows():
    filename = '../data/interim/augmented_'+row['file'][16:-6]+'.pkl'
    embedding_tensor_raw, embedding_tensor_bg, embedding_tensor_fg, mps_raw, mps_bg, mps_fg, df_indices_raw, df_indices_bg, df_indices_fg, vggish_embedding_raw, vggish_embedding_bg, vggish_embedding_fg = data_preprocessing(filename)
    
    embedding_raw_list.append(embedding_tensor_raw)
    embedding_bg_list.append(embedding_tensor_bg)
    embedding_fg_list.append(embedding_tensor_fg)
    mps_raw_list.append(mps_raw)
    mps_bg_list.append(mps_bg)
    mps_fg_list.append(mps_fg)
    indices_raw_list.append(df_indices_raw)
    indices_bg_list.append(df_indices_bg)
    indices_fg_list.append(df_indices_fg)
    vgg_raw_list.append(vggish_embedding_raw)
    vgg_bg_list.append(vggish_embedding_bg)
    vgg_fg_list.append(vggish_embedding_fg)

    
print('seconds: '+str(time.time()-start_time))


seconds: 5145.981775045395


In [9]:
embedding_raw_matrix = np.stack(embedding_raw_list, axis=0)
embedding_bg_matrix = np.stack(embedding_bg_list, axis=0)
embedding_fg_matrix = np.stack(embedding_fg_list, axis=0)

In [10]:
df_indices_raw = pd.concat(indices_raw_list, ignore_index=True)
df_indices_bg = pd.concat(indices_bg_list, ignore_index=True)
df_indices_fg = pd.concat(indices_fg_list, ignore_index=True)

In [11]:
mps_raw_matrix = np.stack(mps_raw_list)
mps_bg_matrix = np.stack(mps_bg_list)
mps_fg_matrix = np.stack(mps_fg_list)

In [12]:
vgg_raw_matrix = np.stack(vgg_raw_list, axis=0)
vgg_bg_matrix = np.stack(vgg_bg_list, axis=0)
vgg_fg_matrix = np.stack(vgg_fg_list, axis=0)

In [13]:
np.save('../data/processed/20230310_dataAugmentation/embedding_raw_matrix.npy', embedding_raw_matrix)
np.save('../data/processed/20230310_dataAugmentation/embedding_bg_matrix.npy', embedding_bg_matrix)
np.save('../data/processed/20230310_dataAugmentation/embedding_fg_matrix.npy', embedding_fg_matrix)

np.save('../data/processed/20230310_dataAugmentation/vgg_raw_matrix.npy', vgg_raw_matrix)
np.save('../data/processed/20230310_dataAugmentation/vgg_bg_matrix.npy', vgg_bg_matrix)
np.save('../data/processed/20230310_dataAugmentation/vgg_fg_matrix.npy', vgg_fg_matrix)

np.save('../data/processed/20230310_dataAugmentation/mps_raw_matrix.npy', mps_raw_matrix)
np.save('../data/processed/20230310_dataAugmentation/mps_bg_matrix.npy', mps_bg_matrix)
np.save('../data/processed/20230310_dataAugmentation/mps_fg_matrix.npy', mps_fg_matrix)

df_indices_raw.to_csv('../data/processed/20230310_dataAugmentation/df_indices_raw.csv')
df_indices_bg.to_csv('../data/processed/20230310_dataAugmentation/df_indices_bg.csv')
df_indices_fg.to_csv('../data/processed/20230310_dataAugmentation/df_indices_fg.csv')